In [35]:
import pandas as pd

df = pd.DataFrame(list(Rating.objects.values_list('user_id', 'work_id', 'choice')), columns=('user_id', 'work_id', 'choice'))

In [7]:
from collections import defaultdict
mapping = defaultdict(lambda: 1)
mapping['willsee'] = 0
mapping['wontsee'] = 0

In [9]:
df['rating'] = df['choice'].map(mapping)

In [10]:
df.head()

,user_id,work_id,choice,rating
0,169,1,like,1
1,1,1146,like,1
2,177,8083,like,1
3,181,1864,willsee,0
4,188,9,like,1


In [12]:
from sklearn.model_selection import train_test_split

In [24]:
encode_user = dict(zip(df['user_id'].unique(), range(3000)))

In [27]:
encode_work = dict(zip(df['work_id'].unique(), range(20000)))

In [28]:
df['encoded_user_id'] = df['user_id'].map(encode_user)
df['encoded_work_id'] = df['work_id'].map(encode_work)

In [34]:
df['encoded_user_id'].max() + 1, df['encoded_work_id'].max() + 1

(2129, 11399)

In [29]:
train, test = train_test_split(df, test_size=0.2)

In [32]:
train[['encoded_user_id', 'encoded_work_id', 'rating']].to_csv('/tmp/train.csv', header=False, index=False)

In [33]:
test[['encoded_user_id', 'encoded_work_id', 'rating']].to_csv('/tmp/test.csv', header=False, index=False)